In [30]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_curve, auc, make_scorer, average_precision_score
import pandas as pd
import numpy as np

load dataset (edit accordingly)

In [31]:
def read_data(str):
    df = pd.read_csv(str)
    print("finished reading")

    label = df.iloc[:,1] 
    datas = df.iloc[:,2:]

    zeros = datas[label == 0]
    ones = datas[label > 0]
    zerolabel = label[label == 0]
    onelabel = label[label > 0]

    train_0,test_0,train_0l,test_0l = train_test_split(zeros, zerolabel, test_size=0.998)

    data_undersample = pd.concat([train_0, ones], axis=0)
    label_undersample = pd.concat([train_0l, onelabel], axis=0)

    final_data = pd.concat([data_undersample, label_undersample], axis=1)
    final_data = shuffle(final_data)

    return final_data

final_df = read_data("all_norm.csv")
print(final_df)

finished reading
                    6             7        8         9       10         11  \
1762198  1.157409e-11  1.157409e-11  0.37774  0.622260  0.37774       0.00   
4047335  7.763263e-09  7.763263e-09  0.37774  0.622260  0.37774       0.00   
1515656  1.568094e-10  1.568094e-10  0.37774  0.622259  0.37774 -934077.12   
3586364  2.429059e-09  2.429059e-09  0.37774  0.622260  0.37774    1316.41   
3208197  3.507589e-12  3.507589e-12  0.37774  0.622260  0.37774     -81.45   
...               ...           ...      ...       ...      ...        ...   
4556949  7.258418e-09  7.258418e-09  0.37774  0.622260  0.37774       0.00   
3959195  3.708957e-09  3.708957e-09  0.37774  0.622260  0.37774   -1927.85   
200813   1.824138e-10  1.824138e-10  0.37774  0.622260  0.37774  -70996.01   
2100628  2.202136e-10  2.202136e-10  0.37774  0.622260  0.37774     195.91   
1250689  2.327243e-09  2.327243e-09  0.37774  0.622260  0.37774    2150.94   

          12            13   14   15  ...   42

In [32]:
X = final_df.iloc[:,1:-1]
y = final_df.iloc[:,-1]
y = pd.Categorical(y)
train_X, test_X, train_y, test_y = train_test_split(X,y)

fit model with grid search hyperparameter

In [33]:
abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

parameters = {'base_estimator__max_depth':[i for i in range(2,11,2)],
              'base_estimator__min_samples_leaf':[5,10],
              'n_estimators':[10,50,250,1000],
              'learning_rate':[0.01,0.1]}

classifier = GridSearchCV(abc, parameters,verbose=3,scoring=make_scorer(average_precision_score, greater_is_better=True),n_jobs=-1)

classifier.fit(train_X, train_y)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier()),
             n_jobs=-1,
             param_grid={'base_estimator__max_depth': [2, 4, 6, 8, 10],
                         'base_estimator__min_samples_leaf': [5, 10],
                         'learning_rate': [0.01, 0.1],
                         'n_estimators': [10, 50, 250, 1000]},
             scoring=make_scorer(average_precision_score), verbose=3)

make prediction and evaluate

In [36]:
# ​predictions = classifier.predict(test_X)
predictions = classifier.predict(test_X)

In [38]:
print(classifier.best_params_)
confusion_matrix(test_y, predictions)

{'base_estimator__max_depth': 4, 'base_estimator__min_samples_leaf': 10, 'learning_rate': 0.01, 'n_estimators': 1000}


array([[2310,  212],
       [ 168, 1141]], dtype=int64)